In [1]:
from typing import Generator
from copy import deepcopy

In [23]:
class BinStr:
    def __init__(self, v="", *, lpad=0, rpad=0):
        if isinstance(v, str):
            assert all(i in {"", "0", "1"} for i in v)
            self.v = "".ljust(lpad, "0") + v + "".rjust(rpad, "0")
        elif isinstance(v, Generator):
            self.v = "".join(map(str, v))

    def __repr__(self):
        return self.v

    def __len__(self):
        return len(self.v)

    def __iter__(self):
        yield from map(int, self.v)

    def __getitem__(self, x):
        return BinStr(self.v[x])

    def __reversed__(self):
        return BinStr(self.v[::-1])

    def __invert__(self):
        return BinStr(1 - a for a in self)

    def __or__(self, other):
        return BinStr(self.v + other.v)

    def __lshift__(self, other):
        temp, self.v = self[:other], repr(self[other:])
        return temp

    def __int__(self):
        return sum(int(repr(self[i])) * 2 ** (len(self) - i - 1) for i in range(len(self)))

    def __add__(self, other):
        assert len(self) == len(other)

        carry = 0
        z = [None] * len(self)

        gen = (a + b + carry for a, b in zip(*map(reversed, [self, other])))

        for i, w in enumerate(gen, start=1):
            carry, z[-i] = divmod(w, 2)

        if carry:
            z.insert(0, 1)

        return BinStr(_ for _ in z)

    def __xor__(self, other):
        return BinStr(abs(a - b) for a, b in zip(self, other))

    def __truediv__(self, other):
        """
        101 11010100
            101
             111
             101
              100
              101
                110
                101
                 110
                 101
                  11
        """
        D = deepcopy(self)
        R = BinStr()  # CRC bits
        offset = len(other)

        wnd_base = len(other)
        print(other, D)

        while D:
            R |= D << offset

            if wnd_base != len(other):
                print(" " * wnd_base, R)

            R ^= other
            R.v = R.v.lstrip("0")

            print(" " * wnd_base, other)
            print(" " * wnd_base, "-" * len(other))

            offset = len(other) - len(R)
            wnd_base += offset

        R = BinStr(R.v, lpad=offset - 1)
        print(" " * wnd_base, R)
        del D
        return R

In [24]:
D = BinStr("10011110")
G = BinStr("1001")

r = len(G) - 1

R = (D | BinStr("0" * r)) / G
print()
(D | R) / G

1001 10011110000
     1001
     ----
         1110
         1001
         ----
          1110
          1001
          ----
           1110
           1001
           ----
            1110
            1001
            ----
             111

1001 10011110111
     1001
     ----
         1110
         1001
         ----
          1111
          1001
          ----
           1101
           1001
           ----
            1001
            1001
            ----
                000


000

In [4]:
def test(actual, expected):
    assert repr(actual) == expected

test(~BinStr("10"), "01")
test(BinStr("11010") + BinStr("11100"), "110110")
test(BinStr("1010") ^ BinStr("0011"), "1001")

test(reversed(BinStr("0011")), "1100")

In [5]:
a = BinStr("1111010011010110")
b = BinStr("1101010011110010")

In [6]:
c = a + b

In [7]:
if len(c) > len(a):
    c = c[1:] + BinStr("1", lpad=len(c) - 2)

~c

0011011000110110